In [11]:
import numpy
import websockets
import json     
import pandas as pd
import asyncio
import ta
from binance.client import Client

%run Keys.py #* our binance keys 

client = Client(api_key,secret_key)

In [3]:
stream = websockets.connect('wss://stream.binance.com:9443/stream?streams=adausdt@miniTicker')
            

In [4]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','c']]
    df.columns =['symbol','Time','Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit= 'ms')

    return df

In [5]:
df = pd.DataFrame()
open_position = False

async with stream as receiver:
    while True:
        data = await receiver.recv()
        data = json.loads(data)['data']
        df = df.append(createframe(data))
        if len(df) > 30:
            if not open_position:
                #* Para abrir posición: el roc en determinado instante debe ser mayor a 0 y el roc inmediato anterior 
                #* no debe estar por encima de cero
                if ta.momentum.roc(df.Price,30).iloc[-1]>0 and \
                    ta.momentum.roc(df.Price,30).iloc[-2]: 

                    order = client.create_order(symbol='ADAUSDT', side='BUY', type='MARKET',quantity='50')
                    print(order)
                    open_position= True
                    buyprice = float(order['fills'][0]['price'])
            if open_position:        
                subdf = df[df.Time>= pd.to_datetime(order['transactTime'],unit='ms')] # only the dataframe after we bought the asset. The first value is gotten at the transactTime
                if len(subdf) >1: #decide after when a new value get into the newdataset
                    subdf['highest'] = subdf.Price.cummax() # cumulative max function
                    subdf['traili ngstop'] = subdf['highest']*0.995
                    if subdf.iloc[-1].Price < subdf.iloc[-1].trailingstop or \
                        df.iloc[-1].Price / buyprice >1.002:
                        order = client.create_order(symbol='ADAUSDT', side='SELL', type='MARKET',quantity='50')
                        print(order)
                        sellprice = float(order['fills'][0]['price'])
                        print(f"You made {(sellprice-buyprice)/buyprice} profit")
                        open_position= False
            print(df.iloc[-1]) 

CancelledError: 